# 1. electrical_data

## 1.0 Startup

In [1]:
disp ('executing gpml startup script...')
mydir = fileparts (mfilename ('fullpath'));                 % where am I located
addpath (mydir)
dirs = {'cov','doc','inf','lik','mean','prior','util'};           % core folders
for d = dirs, addpath (fullfile (mydir, d{1})), end
dirs = {{'util','minfunc'},{'util','minfunc','compiled'}};     % minfunc folders
for d = dirs, addpath (fullfile (mydir, d{1}{:})), end
% addpath([mydir,'/util/sparseinv'])
pkg load statistics

executing gpml startup script...


In [8]:
dataset = 'ele_data'
load ('./data/electrical_data.mat');
[ndata, ~] = size(M);
X = M(:,1:9);
Y = M(:,10);
R = randperm(ndata);        
xt = X(R(1:2000),:);
yt = Y(R(1:2000));
R(1:2000) = [];
x1 = X(R,:);          
y1 = Y(R);
x = x1(1:7000,:);
y = y1(1:7000);
xs = x1(7001:8000,:);
ys = y1(7001:8000,:);
n = 7000; ns = 1000; nt = 2000;   %Ms = 10

dataset = ele_data


In [9]:
sf2 = 1 ; ell = 1 ; sn2 = 0.1 ; 
d = size(x,2);
hyp.cov = log([ones(d,1)*ell;sf2]); hyp.lik = log(sn2); hyp.mean = [];
opts.Xnorm = 'N' ; opts.Ynorm = 'N' ;
opts.Ms = 30 ;
opts.ell = ell ; opts.sf2 = sf2 ; opts.sn2 = sn2 ;
opts.meanfunc = []; opts.covfunc = @covSEard; opts.likfunc = @likGauss; opts.inffunc = @infGaussLik ;

In [11]:
partitionCriterion = 'kmeans'; %, 'kmeans', 'knkmeans'
opts.numOptFC = 25 ;
opts.partitionCriterion = partitionCriterion ;
[models,t_dGP_train] = aggregation_train(x,y,opts) ;
save('./res/elemodel30', 'models','t_dGP_train')

Linesearch     25;  Value 9.878469e+02


partitionCriterion = 'kmeans'; %, 'kmeans', 'knkmeans'
opts.numOptFC = 25 ;
opts.partitionCriterion = partitionCriterion ;
s = load('./res/bikemodel30');
models = s.models;


In [12]:
criterions = {'BCM','PoE'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing ele_data kmeans BCM
BCM: SMSE 0.09406687, MSLL 12.6555, NLPD 12.7736
Processing ele_data kmeans PoE
PoE: SMSE 0.07413277, MSLL 12.4916, NLPD 12.6096


## 1.1 RBCM vs TERBCM

### 1.1.1 RBCM baseline

In [13]:
criterions = {'RBCM'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing ele_data kmeans RBCM
RBCM: SMSE 0.09354422, MSLL 9.2924, NLPD 9.4104


### 1.1.2 Grid q

In [16]:
partitionCriterions = {'kmeans'}; % 'random', 'kmeans', 'knkmeans'
criterions = {'TERBCM'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:6.0;

for i = 1:length(partitionCriterions)
    partitionCriterion = partitionCriterions{i};
    for j = 1:length(criterions)     
        criterion = criterions{j};
        printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
        opts.partitionCriterion = partitionCriterion ;
        %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
        for q = qs
            [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
            [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
            printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
            filename = sprintf('./res/terbcm%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
            save(filename, 'SMSE','MSLL','NLPD')
        endfor
    endfor
endfor


Processing ele_data kmeans TERBCM
TERBCM: SMSE 0.09481171, MSLL 23.5594, NLPD 23.6774 - q   0.20
TERBCM: SMSE 0.09438627, MSLL 15.9417, NLPD 16.0598 - q   0.40
TERBCM: SMSE 0.09405872, MSLL 12.5916, NLPD 12.7096 - q   0.60
TERBCM: SMSE 0.09378366, MSLL 10.6175, NLPD 10.7356 - q   0.80
TERBCM: SMSE 0.09354422, MSLL 9.2924, NLPD 9.4104 - q   1.00
TERBCM: SMSE 0.09333219, MSLL 8.3352, NLPD 8.4532 - q   1.20
TERBCM: SMSE 0.09314293, MSLL 7.6113, NLPD 7.7293 - q   1.40
TERBCM: SMSE 0.09297361, MSLL 7.0470, NLPD 7.1650 - q   1.60
TERBCM: SMSE 0.09282234, MSLL 6.5980, NLPD 6.7160 - q   1.80
TERBCM: SMSE 0.09268778, MSLL 6.2360, NLPD 6.3541 - q   2.00
TERBCM: SMSE 0.09256895, MSLL 5.9420, NLPD 6.0601 - q   2.20
TERBCM: SMSE 0.09246505, MSLL 5.7025, NLPD 5.8206 - q   2.40
TERBCM: SMSE 0.09237541, MSLL 5.5079, NLPD 5.6259 - q   2.60
TERBCM: SMSE 0.09229942, MSLL 5.3509, NLPD 5.4689 - q   2.80
TERBCM: SMSE 0.09223651, MSLL 5.2261, NLPD 5.3442 - q   3.00
TERBCM: SMSE 0.09218611, MSLL 5.1295, NLPD 

### 1.1.3 Find single q

In [21]:
criterion = 'TERBCM';

In [22]:
printf('Optimizing q\r\n');
q1 = grad_q(models, criterion, opts, iter=1000, lr=1e-7, lambda=1e6, init_q=4.0)
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q1) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

Optimizing q
Start preprocessing...
Finish preprocessing...
Optimizing q: remain iter - 1000, q - 4.0000
q1 =  4.0138
TERBCM: SMSE 0.09209855, MSLL 4.9682, NLPD 5.0863


### 1.1.4 Find multi q

In [23]:
criterion = 'TERBCM';

In [28]:
printf('Optimizing q\r\n');
iqs = 4.0 + 0.01 * rand(opts.Ms,1)
for ldk=-1:3
    ld = 10^ldk
    qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e0, lambda=ld, init_q=iqs)
    [mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
endfor

Optimizing q
iqs =

   4.0048
   4.0079
   4.0006
   4.0016
   4.0016
   4.0032
   4.0014
   4.0087
   4.0087
   4.0061
   4.0060
   4.0081
   4.0092
   4.0026
   4.0064
   4.0075
   4.0039
   4.0061
   4.0073
   4.0089
   4.0094
   4.0029
   4.0039
   4.0082
   4.0034
   4.0059
   4.0039
   4.0062
   4.0027
   4.0095

ld =  0.10000
Start preprocessing...
k =  1
k =  2
k =  3
k =  4
k =  5
k =  6
k =  7
k =  8
k =  9
k =  10
k =  11
k =  12
k =  13
k =  14
k =  15
k =  16
k =  17
k =  18
k =  19
k =  20
k =  21
k =  22
k =  23
k =  24
k =  25
k =  26
k =  27
k =  28
k =  29
k =  30
Finish preprocessing...
qs =

   3.9875
   4.0039
   3.9684
   3.9703
   3.9800
   3.9751
   3.9765
   3.9890
   3.9994
   3.9970
   3.9969
   3.9922
   4.0009
   3.9773
   4.0015
   3.9911
   3.9841
   3.9938
   3.9800
   3.9959
   3.9771
   3.9880
   3.9893
   3.9978
   3.9923
   3.9822
   3.9775
   3.9871
   3.9808
   4.0021

TERBCM: SMSE 0.09209869, MSLL 4.9684, NLPD 5.0865
ld =  1
Start preprocessing...

## 1.2 GRBCM vs TEGRBCM

### 1.2.1 GRBCM baseline

In [29]:
criterions = {'GRBCM'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing ele_data kmeans GRBCM
GRBCM: SMSE 0.07481737, MSLL 0.0046, NLPD 0.1226


### 1.2.2 Grid q

In [30]:
criterions = {'TEGRBCM'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:4.0;

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    for q = qs
        [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
        [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
        printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
        filename = sprintf('./res/tegrbcm%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
        save(filename, 'SMSE','MSLL','NLPD')
    endfor
endfor

Processing ele_data kmeans TEGRBCM
TEGRBCM: SMSE 0.07480678, MSLL 0.0045, NLPD 0.1225 - q   0.20
TEGRBCM: SMSE 0.07481311, MSLL 0.0045, NLPD 0.1226 - q   0.40
TEGRBCM: SMSE 0.07481556, MSLL 0.0045, NLPD 0.1226 - q   0.60
TEGRBCM: SMSE 0.07481676, MSLL 0.0046, NLPD 0.1226 - q   0.80
TEGRBCM: SMSE 0.07481737, MSLL 0.0046, NLPD 0.1226 - q   1.00
TEGRBCM: SMSE 0.07481764, MSLL 0.0046, NLPD 0.1226 - q   1.20
TEGRBCM: SMSE 0.07481768, MSLL 0.0046, NLPD 0.1226 - q   1.40
TEGRBCM: SMSE 0.07481756, MSLL 0.0046, NLPD 0.1226 - q   1.60
TEGRBCM: SMSE 0.07481732, MSLL 0.0046, NLPD 0.1226 - q   1.80
TEGRBCM: SMSE 0.07481696, MSLL 0.0046, NLPD 0.1226 - q   2.00
TEGRBCM: SMSE 0.07481652, MSLL 0.0046, NLPD 0.1226 - q   2.20
TEGRBCM: SMSE 0.07481599, MSLL 0.0045, NLPD 0.1226 - q   2.40
TEGRBCM: SMSE 0.07481538, MSLL 0.0045, NLPD 0.1226 - q   2.60
TEGRBCM: SMSE 0.07481470, MSLL 0.0045, NLPD 0.1226 - q   2.80
TEGRBCM: SMSE 0.07481394, MSLL 0.0045, NLPD 0.1226 - q   3.00
TEGRBCM: SMSE 0.07481310, MSLL 0.00

### 1.2.3 Find single q

In [31]:
criterion = 'TEGRBCM';

In [32]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=1000, lr=1e-9, lambda=1e6, init_q=2.0);

Optimizing q
Start preprocessing...
Finish preprocessing...
Optimizing q: remain iter - 1000, q - 2.0000


In [33]:
q

q =  2.0000


In [34]:
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

### 1.2.4 Find multi q

In [35]:
criterion = 'TEGRBCM';

In [38]:
printf('Optimizing q\r\n');
iqs = 2.0 + 0.01 * rand(opts.Ms,1)
for ldk=-1:3
    ld = 10^ldk
    qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e3, lambda=ld, init_q=iqs)
    [mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
endfor

Optimizing q
iqs =

   2.0055
   2.0072
   2.0066
   2.0052
   2.0087
   2.0067
   2.0089
   2.0031
   2.0048
   2.0085
   2.0092
   2.0043
   2.0043
   2.0092
   2.0094
   2.0015
   2.0031
   2.0090
   2.0097
   2.0056
   2.0031
   2.0041
   2.0069
   2.0020
   2.0025
   2.0058
   2.0052
   2.0008
   2.0080
   2.0025

ld =  0.10000
Start preprocessing...
k =  1
k =  2
k =  3
k =  4
k =  5
k =  6
k =  7
k =  8
k =  9
k =  10
k =  11
k =  12
k =  13
k =  14
k =  15
k =  16
k =  17
k =  18
k =  19
k =  20
k =  21
k =  22
k =  23
k =  24
k =  25
k =  26
k =  27
k =  28
k =  29
k =  30
Finish preprocessing...
qs =

   2.0055
   2.0072
   2.0032
   2.0055
   1.9964
   2.0057
   2.0097
   2.0046
   2.0059
   1.9944
   2.0024
   2.0011
   1.9966
   2.0063
   2.0013
   1.9958
   2.0034
   2.0029
   2.0109
   1.9984
   2.0005
   2.0009
   1.9989
   1.9996
   2.0033
   2.0013
   1.9925
   1.9948
   2.0075
   1.9974

TEGRBCM: SMSE 0.07481696, MSLL 0.0046, NLPD 0.1226
ld =  1
Start preprocessing..

## 1.3 GPoE vs TEGPoE

### 1.3.1 GPoE baseline

In [39]:
criterions = {'GPoE'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing ele_data kmeans GPoE
GPoE: SMSE 0.07413276, MSLL 9.1719, NLPD 9.2899


### 1.3.2 Grid q

In [41]:
partitionCriterions = {'kmeans'}; % 'random', 'kmeans', 'knkmeans'
criterions = {'TEGPoE'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:6.0;

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    for q = qs
        [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
        [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
        printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
        filename = sprintf('./res/tegpoe%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
        save(filename, 'SMSE','MSLL','NLPD')
    endfor
endfor

Processing ele_data kmeans TEGPoE
TEGPoE: SMSE 0.07413276, MSLL 23.2562, NLPD 23.3742 - q   0.20
TEGPoE: SMSE 0.07413276, MSLL 15.7357, NLPD 15.8537 - q   0.40
TEGPoE: SMSE 0.07413276, MSLL 12.4285, NLPD 12.5465 - q   0.60
TEGPoE: SMSE 0.07413276, MSLL 10.4799, NLPD 10.5979 - q   0.80
TEGPoE: SMSE 0.07413276, MSLL 9.1719, NLPD 9.2899 - q   1.00
TEGPoE: SMSE 0.07413276, MSLL 8.2271, NLPD 8.3451 - q   1.20
TEGPoE: SMSE 0.07413276, MSLL 7.5126, NLPD 7.6306 - q   1.40
TEGPoE: SMSE 0.07413276, MSLL 6.9556, NLPD 7.0736 - q   1.60
TEGPoE: SMSE 0.07413275, MSLL 6.5124, NLPD 6.6305 - q   1.80
TEGPoE: SMSE 0.07413275, MSLL 6.1552, NLPD 6.2733 - q   2.00
TEGPoE: SMSE 0.07413275, MSLL 5.8650, NLPD 5.9831 - q   2.20
TEGPoE: SMSE 0.07413275, MSLL 5.6286, NLPD 5.7467 - q   2.40
TEGPoE: SMSE 0.07413275, MSLL 5.4365, NLPD 5.5546 - q   2.60
TEGPoE: SMSE 0.07413275, MSLL 5.2816, NLPD 5.3997 - q   2.80
TEGPoE: SMSE 0.07413275, MSLL 5.1585, NLPD 5.2765 - q   3.00
TEGPoE: SMSE 0.07413275, MSLL 5.0631, NLPD 

### 1.3.3 Find single q

In [ ]:
criterion = 'TEGPoE';

In [42]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=1000, lr=1e-9, lambda=1e6, init_q=4.0);

Optimizing q
Start preprocessing...
Finish preprocessing...
Optimizing q: remain iter - 1000, q - 4.0000


In [43]:
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

### 1.3.4 Find multi q

In [45]:
printf('Optimizing q\r\n');
iqs = 4.0 + 0.01 * rand(opts.Ms,1)
for ldk=-1:3
    ld = 10^ldk
    qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e3, lambda=ld, init_q=iqs)
    [mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
endfor

Optimizing q
iqs =

   4.0076
   4.0017
   4.0030
   4.0071
   4.0011
   4.0015
   4.0067
   4.0070
   4.0038
   4.0041
   4.0098
   4.0066
   4.0043
   4.0068
   4.0083
   4.0062
   4.0032
   4.0045
   4.0055
   4.0047
   4.0091
   4.0059
   4.0001
   4.0050
   4.0088
   4.0068
   4.0036
   4.0037
   4.0093
   4.0071

ld =  0.10000
Start preprocessing...
k =  1
k =  2
k =  3
k =  4
k =  5
k =  6
k =  7
k =  8
k =  9
k =  10
k =  11
k =  12
k =  13
k =  14
k =  15
k =  16
k =  17
k =  18
k =  19
k =  20
k =  21
k =  22
k =  23
k =  24
k =  25
k =  26
k =  27
k =  28
k =  29
k =  30
Finish preprocessing...
qs =

   8.0001e-01
   8.0001e-01
   8.0001e-01
   8.0001e-01
   8.0001e-01
   1.0000e-05
   8.0001e-01
   1.0000e-05
   8.0001e-01
   8.0001e-01
   3.1181e-02
   8.0001e-01
   1.0000e-05
   8.0001e-01
   1.8125e+01
   1.0000e-05
   1.0000e-05
   8.0001e-01
   1.0000e-05
   8.0001e-01
   1.0000e-05
   1.0000e-05
   8.0001e-01
   1.0000e-05
   1.0000e-05
   8.0001e-01
   8.0001e-01
   